In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model

2025-03-04 12:49:17.210297: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-04 12:49:17.220374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741072757.231983   25901 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741072757.235590   25901 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-04 12:49:17.246996: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# ==========================================================
# Step 1: Define the Custom RLRR Layer (Adaptation Module)
# ==========================================================
class RLRRDense(layers.Layer):
    def __init__(self, units, use_bias=True, kernel_initializer='glorot_uniform', **kwargs):
        super(RLRRDense, self).__init__(**kwargs)
        self.units = units
        self.use_bias = use_bias
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        
    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        self.W_frozen = self.add_weight(
            shape=(input_dim, self.units),
            initializer=self.kernel_initializer,
            trainable=False,
            name="W_frozen"
        )
        self.s_left = self.add_weight(
            shape=(input_dim, 1),
            initializer='zeros',
            trainable=True,
            name="s_left"
        )
        self.s_right = self.add_weight(
            shape=(1, self.units),
            initializer='zeros',
            trainable=True,
            name="s_right"
        )
        self.f = self.add_weight(
            shape=(input_dim, self.units),
            initializer='zeros',
            trainable=True,
            name="f_bias"
        )
        if self.use_bias:
            self.bias = self.add_weight(
                shape=(self.units,),
                initializer="zeros",
                trainable=True,
                name="bias"
            )
        else:
            self.bias = None
        super(RLRRDense, self).build(input_shape)
    
    def call(self, inputs):
        scaling = 1 + tf.matmul(self.s_left, self.s_right)
        W_reparam = scaling * self.W_frozen + self.f
        output = tf.matmul(inputs, W_reparam)
        if self.use_bias:
            output = tf.nn.bias_add(output, self.bias)
        return output


In [ ]:
# ==========================================================
# Step 2: Load a Pre-trained ViT Backbone from TensorFlow Hub
# ==========================================================
vit_url = "https://tfhub.dev/sayakpaul/vit_b16_fe/1"
vit_layer = hub.KerasLayer(vit_url, trainable=False)


In [ ]:
# ==========================================================
# Step 3: Build the Fine-Tuning Model
# ==========================================================
input_img = tf.keras.Input(shape=(224, 224, 3))

# Normalize input
x = layers.Rescaling(1./255.0)(input_img)

# ✅ FIX: Wrap `hub.KerasLayer` in a Lambda layer to prevent symbolic tensor errors
features = layers.Lambda(lambda img: vit_layer(img))(x)

# Add RLRR adaptation layer
x = RLRRDense(256, use_bias=True)(features)
x = layers.ReLU()(x)

# Add a classification head
output = layers.Dense(10, activation='softmax')(x)

# Construct the full model
model = Model(inputs=input_img, outputs=output)
model.summary()

In [ ]:
# ==========================================================
# Step 4: Load and Prepare the Dataset Efficiently
# ==========================================================
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert labels to one-hot encoding
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

# ✅ Efficient Data Pipeline (No OOM Errors)
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_image(image, label):
    image = tf.image.resize(tf.cast(image, tf.float32), (224, 224))  # Resize dynamically
    return image, label

train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train_cat))
            .map(preprocess_image, num_parallel_calls=AUTOTUNE)
            .batch(64)
            .prefetch(AUTOTUNE))

test_ds = (tf.data.Dataset.from_tensor_slices((x_test, y_test_cat))
           .map(preprocess_image, num_parallel_calls=AUTOTUNE)
           .batch(64)
           .prefetch(AUTOTUNE))


In [ ]:
# ==========================================================
# Step 5: Compile and Train the Model
# ==========================================================
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history =model.fit(train_ds,
          validation_data=test_ds,
          epochs=1)


In [ ]:
# ==========================================================
# Step 6: Evaluate the Model
# ==========================================================
loss, accuracy = model.evaluate(test_ds)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Save model
model.save("vit_finetuned_rlrr_model.h5")